# 1. IMPORTS

## 1.1 Libraries

In [3]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from datetime import datetime

## 1.2 Raw data

In [5]:
#import imdb dataset 
df = pd.read_csv("../data/TMDB_movie_dataset_v11.csv", encoding='utf-8')

#import preprocessed cmu dataset 
df_movies=pd.read_csv("../temporary/preprocessed_data.csv",index_col = 'Wikipedia movie ID')
nan_counts_before = df_movies.isna().sum()
print(nan_counts_before)

Movie name                      0
Movie release month         42365
Movie release year           6901
Movie release date          42365
Movie box office revenue    73337
Movie runtime               20450
Movie countries                 0
Movie genres                    0
dtype: int64


In [6]:
#import summaries
PLOT_SUMMARIES = "../data/plot_summaries.txt"
with open(PLOT_SUMMARIES, 'r', encoding='utf-8') as file:
    plot_summaries = file.readlines()

data = []
for line in plot_summaries:
    parts = line.strip().split('\t', 1) # Split each line by the tab character
    if len(parts) == 2:
        movie_id, summary = parts
        data.append([movie_id, summary])

df_summary = pd.DataFrame(data, columns=['ID', 'Summary'])
df_summary.set_index('ID', inplace=True)



In [7]:
df_summary.sample(2)

,Summary
ID,
34195557,The story deals with two friends Kishorand Eka...
19360183,It is a windy spring night. A man tells Kenned...


In [8]:
df.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [9]:
df.dtypes

id                        int64
title                    object
vote_average            float64
vote_count                int64
status                   object
release_date             object
revenue                   int64
runtime                   int64
adult                      bool
backdrop_path            object
budget                    int64
homepage                 object
imdb_id                  object
original_language        object
original_title           object
overview                 object
popularity              float64
poster_path              object
tagline                  object
genres                   object
production_companies     object
production_countries     object
spoken_languages         object
keywords                 object
dtype: object

In [10]:
#drop duplicate movies
print(df["id"].is_unique)
df = df.drop_duplicates(subset="id", keep="first")
print(df["id"].is_unique)


False
True


# 2. DATA SELECTION & PREPARATION

In [12]:
#keep only the relevant columns title and release date for matching
df_relevant = df[['title', 'release_date','revenue','overview','budget']].copy(deep = True)

In [13]:
#change date column to date format
df_relevant['release_date'] = pd.to_datetime(df_relevant['release_date'], format='%Y-%m-%d')
print(df_relevant['release_date'].isna().sum())

# Add a new column 'year' with the year extracted from the date. Those will help for matching with the CMU set. 
df_relevant['year'] = df_relevant['release_date'].dt.year
df_relevant['month'] = df_relevant['release_date'].dt.month

print(df_relevant['year'].isna().sum())
print(df_relevant['month'].isna().sum())

#Replace all zero budget and revenues by np.nan (when data isn't available)
df_relevant['revenue'] = df_relevant['revenue'].replace(0, np.nan)

df_relevant['budget'] = df_relevant['budget'].replace(0, np.nan)



185133
185133
185133


In [14]:
# remove duplicates of movies that have different ID's

#print the number of duplicates after
duplicates_relevant = df_relevant.duplicated(subset=['year', 'title']).sum()
print(f"Duplicates in df_relevant: {duplicates_relevant}")

duplicates_movies = df_movies.duplicated(subset=['Movie release year', 'Movie name']).sum()
print(f"Duplicates in df_movies: {duplicates_movies}")

df_movies = df_movies.drop_duplicates(subset=['Movie release year', 'Movie name'], keep="first")

df_relevant = df_relevant.drop_duplicates(subset=['year', 'title'], keep="first")


#print the number of duplicates after
duplicates_relevant = df_relevant.duplicated(subset=['year', 'title']).sum()
print(f"Duplicates in df_relevant: {duplicates_relevant}")

duplicates_movies = df_movies.duplicated(subset=['Movie release year', 'Movie name']).sum()
print(f"Duplicates in df_movies: {duplicates_movies}")

Duplicates in df_relevant: 27344
Duplicates in df_movies: 187
Duplicates in df_relevant: 0
Duplicates in df_movies: 0


In [15]:
#Get shape of both datasets, IMDB has a lot more movies. However, we merge on CMU as we are forced to use this dataset.
#Note we could use the IMDB dataset only that contains (near to) everything from the CMU dataset.
print(df_relevant.shape)
print(df_movies.shape)

(1105379, 7)
(81550, 8)


# 3. DATA MERGING

In [17]:
#add summaries to CMU dataset
df_movies = pd.merge(df_movies, df_summary, left_index=True, right_index=True, how='left')



#Merge both datasets
df_merged = pd.merge(
    df_movies, 
    df_relevant,
    left_on=['Movie release year', 'Movie name'],
    right_on=['year', 'title'],
    how='left')

#have the same indices as the CMU dataset
df_merged.index = df_movies.index
print(df_merged.shape)

df_merged.head(5)


(81550, 16)


,Movie name,Movie release month,Movie release year,Movie release date,Movie box office revenue,Movie runtime,Movie countries,Movie genres,Summary,title,release_date,revenue,overview,budget,year,month
Wikipedia movie ID,,,,,,,,,,,,,,,,
975900,Ghosts of Mars,8.0,2001.0,2001-08-24,2.477118e+07,98.0,['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...",NaN,Ghosts of Mars,2001-08-24,14010832.0,"In 2176, a Martian police unit is sent to pick...",28000000.0,2001.0,8.0
3196793,Getting Away with Murder: The JonBenét Ramsey ...,2.0,2000.0,2000-02-16,NaN,95.0,['United States of America'],"['Mystery', 'Biographical film', 'Drama', 'Cri...",NaN,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,Dramatization of the story behind the murder o...,NaN,2000.0,2.0
28463795,Brun bitter,NaN,1988.0,NaN,NaN,83.0,['Norway'],"['Crime Fiction', 'Drama']",NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
9363483,White Of The Eye,NaN,1987.0,NaN,NaN,110.0,['United Kingdom'],"['Thriller', 'Erotic thriller', 'Psychological...",NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
261236,A Woman in Flames,NaN,1983.0,NaN,NaN,106.0,['Germany'],['Drama'],NaN,A Woman in Flames,1983-05-11,NaN,"Eva, an upper-class housewife, frustratedly le...",NaN,1983.0,5.0


# 4. DATA HANDLING

In [19]:
#Box office values are difficult to compare across different years due to inflation and varying years of measurement.
#Adjust all (USD) values for inflation
inflation_data = {
    1914: 1.3, 1915: 0.9, 1916: 7.7, 1917: 17.8, 1918: 17.3, 
    1919: 15.2, 1920: 15.6, 1921: -10.9, 1922: -6.2, 1923: 1.8, 
    1924: 0.4, 1925: 2.4, 1926: 0.9, 1927: -1.9, 1928: -1.2, 
    1929: 0.0, 1930: -2.7, 1931: -8.9, 1932: -10.3, 1933: -5.2, 
    1934: 3.5, 1935: 2.6, 1936: 1.0, 1937: 3.7, 1938: -2.0, 
    1939: -1.3, 1940: 0.7, 1941: 5.1, 1942: 10.9, 1943: 6.0, 
    1944: 1.6, 1945: 2.3, 1946: 8.5, 1947: 14.4, 1948: 7.7, 
    1949: -1.0, 1950: 1.1, 1951: 7.9, 1952: 2.3, 1953: 0.8, 
    1954: 0.3, 1955: -0.3, 1956: 1.5, 1957: 3.3, 1958: 2.7, 
    1959: 1.08, 1960: 1.5, 1961: 1.1, 1962: 1.2, 1963: 1.2, 
    1964: 1.3, 1965: 1.6, 1966: 3.0, 1967: 2.8, 1968: 4.3, 
    1969: 5.5, 1970: 5.8, 1971: 4.3, 1972: 3.3, 1973: 6.2, 
    1974: 11.1, 1975: 9.1, 1976: 5.7, 1977: 6.5, 1978: 7.6, 
    1979: 11.3, 1980: 13.5, 1981: 10.3, 1982: 6.1, 1983: 3.2, 
    1984: 4.3, 1985: 3.5, 1986: 1.9, 1987: 3.7, 1988: 4.1, 
    1989: 4.8, 1990: 5.4, 1991: 4.2, 1992: 3.0, 1993: 3.0, 
    1994: 2.6, 1995: 2.8, 1996: 2.9, 1997: 2.3, 1998: 1.6, 
    1999: 2.2, 2000: 3.4, 2001: 2.8, 2002: 1.6, 2003: 2.3, 
    2004: 2.7, 2005: 3.4, 2006: 3.2, 2007: 2.9, 2008: 3.8, 
    2009: -0.4, 2010: 1.6, 2011: 3.2, 2012: 2.1, 2013: 1.5, 
    2014: 1.6, 2015: 0.1, 2016: 1.3, 2017: 2.1, 2018: 2.4, 
    2019: 1.8, 2020: 1.2, 2021: 4.7, 2022: 8.0, 2023: 4.1, 2024: 3.2
}

#source : minesota website : https://www.minneapolisfed.org/about-us/monetary-policy/inflation-calculator/consumer-price-index-1913-

#Funtion to apply the inflation rates to the column which takes year and amount into account
def adjust_for_inflation(year, amount, inflation_data):
    """
    Adjusts the given amount from the provided year to 2024 based on annual inflation rates.

    Returns the inflation-adjusted amount for 2024.
    """
    adjusted_value = amount
    year = int(year)
    for y in range(year, 2024):
        if y in inflation_data:
            inflation_rate = inflation_data[y]
            # Adjust for inflation for each year
            adjusted_value *= (1 + inflation_rate / 100)
    return adjusted_value


#apply function when year and revenue isn't NaN for revenue
for i in range(len(df_merged)):
    if pd.notna(df_merged.iloc[i]['Movie release year']) and pd.notna(df_merged.iloc[i]['revenue']):
        release_year = df_merged['Movie release year'].iloc[i]
        
        adjusted_revenue = adjust_for_inflation(release_year, df_merged.iloc[i]['revenue'], inflation_data)
        
        df_merged.iloc[i, df_merged.columns.get_loc('revenue')] = adjusted_revenue


#apply function when year and box office isn't NaN for budget
for i in range(len(df_merged)):
    if pd.notna(df_merged.iloc[i]['Movie release year']) and pd.notna(df_merged.iloc[i]['budget']):
        release_year = df_merged['Movie release year'].iloc[i]
        
        adjusted_revenue = adjust_for_inflation(release_year, df_merged.iloc[i]['budget'], inflation_data)
        
        df_merged.iloc[i, df_merged.columns.get_loc('budget')] = adjusted_revenue


df_merged.head(5)

,Movie name,Movie release month,Movie release year,Movie release date,Movie box office revenue,Movie runtime,Movie countries,Movie genres,Summary,title,release_date,revenue,overview,budget,year,month
Wikipedia movie ID,,,,,,,,,,,,,,,,
975900,Ghosts of Mars,8.0,2001.0,2001-08-24,2.477118e+07,98.0,['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...",NaN,Ghosts of Mars,2001-08-24,2.477118e+07,"In 2176, a Martian police unit is sent to pick...",4.950406e+07,2001.0,8.0
3196793,Getting Away with Murder: The JonBenét Ramsey ...,2.0,2000.0,2000-02-16,NaN,95.0,['United States of America'],"['Mystery', 'Biographical film', 'Drama', 'Cri...",NaN,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,Dramatization of the story behind the murder o...,NaN,2000.0,2.0
28463795,Brun bitter,NaN,1988.0,NaN,NaN,83.0,['Norway'],"['Crime Fiction', 'Drama']",NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
9363483,White Of The Eye,NaN,1987.0,NaN,NaN,110.0,['United Kingdom'],"['Thriller', 'Erotic thriller', 'Psychological...",NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
261236,A Woman in Flames,NaN,1983.0,NaN,NaN,106.0,['Germany'],['Drama'],NaN,A Woman in Flames,1983-05-11,NaN,"Eva, an upper-class housewife, frustratedly le...",NaN,1983.0,5.0


In [20]:
#Merge summary and overview columns
df_merged['Summary'] = df_merged['Summary'].fillna(df_merged['overview'])

#Merge Box office and revenue columns
df_merged['Movie box office revenue'] = df_merged['Movie box office revenue'].fillna(df_merged['revenue'])


In [21]:
#merge dates, months and years columns
df_merged['Movie release date'] = pd.to_datetime(df_merged['Movie release date']).dt.strftime('%Y-%m-%d')
df_merged['Movie release month'] = df_merged['Movie release month'].fillna(df_merged['month'])
df_merged['Movie release year'] = df_merged['Movie release year'].fillna(df_merged['year'])
df_merged['Movie release date'] = df_merged['Movie release date'].fillna(df_merged['release_date'])
df_merged['Movie release date'] = pd.to_datetime(df_merged['Movie release date']).dt.strftime('%Y-%m-%d')

#drop the duplicate columns that were merged into the CMU columns
df_merged.drop(columns = ['release_date','year','month','overview','revenue','release_date','title'],inplace = True)

df_merged.head()

,Movie name,Movie release month,Movie release year,Movie release date,Movie box office revenue,Movie runtime,Movie countries,Movie genres,Summary,budget
Wikipedia movie ID,,,,,,,,,,
975900,Ghosts of Mars,8.0,2001.0,2001-08-24,2.477118e+07,98.0,['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...","In 2176, a Martian police unit is sent to pick...",4.950406e+07
3196793,Getting Away with Murder: The JonBenét Ramsey ...,2.0,2000.0,2000-02-16,NaN,95.0,['United States of America'],"['Mystery', 'Biographical film', 'Drama', 'Cri...",Dramatization of the story behind the murder o...,NaN
28463795,Brun bitter,NaN,1988.0,NaN,NaN,83.0,['Norway'],"['Crime Fiction', 'Drama']",NaN,NaN
9363483,White Of The Eye,NaN,1987.0,NaN,NaN,110.0,['United Kingdom'],"['Thriller', 'Erotic thriller', 'Psychological...",NaN,NaN
261236,A Woman in Flames,5.0,1983.0,1983-05-11,NaN,106.0,['Germany'],['Drama'],"Eva, an upper-class housewife, frustratedly le...",NaN


In [22]:
nan_counts = df_merged.isna().sum()
print(nan_counts)

Movie name                      0
Movie release month         21171
Movie release year           6834
Movie release date          21171
Movie box office revenue    70535
Movie runtime               20379
Movie countries                 0
Movie genres                    0
Summary                     32459
budget                      73232
dtype: int64


In [23]:
#How many NaN values did we fill?
nan_counts_before-nan_counts

Movie box office revenue     2802.0
Movie countries                 0.0
Movie genres                    0.0
Movie name                      0.0
Movie release date          21194.0
Movie release month         21194.0
Movie release year             67.0
Movie runtime                  71.0
Summary                         NaN
budget                          NaN
dtype: float64

In [24]:
df_merged.sample(5)

,Movie name,Movie release month,Movie release year,Movie release date,Movie box office revenue,Movie runtime,Movie countries,Movie genres,Summary,budget
Wikipedia movie ID,,,,,,,,,,
21969461,Paris Is Always Paris,11.0,1951.0,1951-11-15,NaN,95.0,"['France', 'Italy']",['Comedy film'],A group of Italian soccer fans arrive in Paris...,NaN
20073084,The Tenth Symphony,11.0,1918.0,1918-11-01,NaN,NaN,['France'],"['Silent film', 'Drama', 'Black-and-white']",Composer Enrid Damor knows nothing of the past...,NaN
20331472,A Home at the End of the World,7.0,2004.0,2004-07-23,2.563062e+06,97.0,['United States of America'],"['LGBT', 'Melodrama', 'Gay', 'Period piece', '...","Three friends form a bond over the year, Johna...",1.075558e+07
15293938,Bloody Beach,8.0,2000.0,2000-08-12,NaN,85.0,['South Korea'],"['Horror', 'Slasher', 'World cinema']",A group of Internet chatters decides to spend ...,NaN
20465601,To Live Again,1.0,1963.0,1963-01-01,NaN,NaN,['United States of America'],"['Short Film', 'Documentary']",To Live Again is a 1963 short documentary film...,NaN


In [25]:
df_merged.to_csv("../temporary/merged_data.csv")

In [26]:
df_merged.shape

(81550, 10)